In [ ]:
!pip install langchain
!pip install langchain-community
!pip install pinecone-client
!pip install langchain_openai
!pip install langchain-pinecone
!pip install unstructured
import uuid
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings
from openai import OpenAI
import os
import glob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 802.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/

initialize

GetSecrets

In [ ]:
from google.colab import userdata
userdata.get('pinecone-ai')
userdata.get('openai-key')

'sk-proj-ojox8TRsqSUWlOuA5h08T3BlbkFJDNOnM2roUxAWupMaH5Ri'

init open ai

In [ ]:
embeddings = OpenAIEmbeddings(api_key=userdata.get('openai-key'), model="text-embedding-ada-002")
openai = OpenAI(api_key=userdata.get('openai-key'))

pc = Pinecone(api_key=userdata.get('pinecone-ai'))


 Get document path

In [ ]:
documentPath = './MarkdownFiles' #add path to md file
index = pc.Index("yogi-index") #Instance name in pinecone

load data and chunk it up MD

In [ ]:
# loader = UnstructuredMarkdownLoader(documentPath)
loader = DirectoryLoader(documentPath, glob="*.md")
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)


Define methods that will be used to upsert

In [ ]:
def generate_uuid():
    return str(uuid.uuid4())

unique_id = generate_uuid()

def generate_embeddings(text):
    response = openai.embeddings.create(
        model="text-embedding-ada-002",
        input=text
    )
    embedding = response.data[0].embedding
    return embedding

def upload_to_pinecone(doc_id, embedding, metadata):
    index.upsert(vectors=[{
        "id": doc_id,
        "values": embedding,
        "metadata": metadata
    }])
    print(f"Document ID {doc_id} with metadata uploaded successfully.")

def process_and_upload_document(document):
    doc_id = str(uuid.uuid4())
    embedding = generate_embeddings(document.page_content)
    upload_to_pinecone(doc_id, embedding, document.metadata)

    return embedding

loop through each chunk and upsert to pinecone.

In [ ]:
for text in texts:
    process_and_upload_document(text)


Document ID 38b01149-b4ec-4f98-8c26-35a32045d202 with metadata uploaded successfully.
Document ID 3199b0ee-8af7-48d2-bb89-77882c7bc313 with metadata uploaded successfully.
Document ID eace4fb7-3fe9-4d9f-b006-38683f248991 with metadata uploaded successfully.
Document ID ccb9ad42-598b-4546-bf72-07c215b18e0c with metadata uploaded successfully.
Document ID 05a13936-33ad-4dc8-972a-38f7e2102ad8 with metadata uploaded successfully.
Document ID d108462f-12ae-423e-9aa2-0d60d2823e5e with metadata uploaded successfully.
Document ID 2f5c8f51-7af6-4dec-9084-fa8c7c00d1b7 with metadata uploaded successfully.
Document ID c14704cf-efd2-4842-8a48-d480138c5145 with metadata uploaded successfully.
Document ID 1c3eb9a8-66a7-4685-9d25-5ab12882250d with metadata uploaded successfully.
Document ID 0a549c3f-3d9b-49fa-b601-312b25e7ed31 with metadata uploaded successfully.
Document ID 7e8487d2-8a1c-420c-b307-ec96e9e380f4 with metadata uploaded successfully.
Document ID aaa3fe4c-d11c-46bf-8021-de0522d848b5 with 